In [ ]:
# The code was removed by Watson Studio for sharing.

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import os

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
np.random.seed(42)

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout # new!
from keras.layers.normalization import BatchNormalization # new!
from keras import regularizers # new! 
from keras.optimizers import SGD

Using TensorFlow backend.


In [4]:
# The code was removed by Watson Studio for sharing.

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:

body = client_3269765585d54624a3f9e9ca7b7a831e.get_object(Bucket='deeplearning-donotdelete-pr-msmmmlhpmtzwze',Key='test_Y3wMUE5_7gLdaTN.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test = pd.read_csv(body)
test.head()



,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [6]:

body = client_3269765585d54624a3f9e9ca7b7a831e.get_object(Bucket='deeplearning-donotdelete-pr-msmmmlhpmtzwze',Key='Sample_Submission_ZAuTl8O_FK3zQHh.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

sample_discussion = pd.read_csv(body)
sample_discussion.head()



,Loan_ID,Loan_Status


In [7]:
train_original=train.copy() 
test_original=test.copy()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
Loan_ID              367 non-null object
Gender               356 non-null object
Married              367 non-null object
Dependents           357 non-null object
Education            367 non-null object
Self_Employed        344 non-null object
ApplicantIncome      367 non-null int64
CoapplicantIncome    367 non-null int64
LoanAmount           362 non-null float64
Loan_Amount_Term     361 non-null float64
Credit_History       338 non-null float64
Property_Area        367 non-null object
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


In [8]:
#fill the missing values using fillna
train['Gender'].fillna(train['Gender'].mode()[0], inplace=True) 
train['Married'].fillna(train['Married'].mode()[0], inplace=True) 
train['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True) 
train['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True) 
train['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)

In [9]:
#Now let’s try to find a way to fill the missing values in Loan_Amount_Term. We will look at the value count of the Loan amount term variable.
train['Loan_Amount_Term'].value_counts()

360.0    512
180.0     44
480.0     15
300.0     13
84.0       4
240.0      4
120.0      3
36.0       2
60.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [10]:
#It can be seen that in loan amount term variable, the value of 360 is repeating the most. So we will replace the missing values in this variable using the mode of this variable.
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)

In [11]:
#Now we will see the LoanAmount variable. As it is a numerical variable, we can use mean or median to impute the missing values. 
#We will use median to fill the null values as earlier we saw that loan amount have outliers so the mean will not be the proper approach as it is highly affected by the presence of outliers.
train['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)

In [12]:
#Now lets check whether all the missing values are filled in the dataset.
train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [13]:
#As we can see that all the missing values have been filled in the test dataset. Let’s fill all the missing values in the test dataset too with the same approach.
test['Gender'].fillna(train['Gender'].mode()[0], inplace=True) 
test['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True) 
test['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True) 
test['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True) 
test['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True) 
test['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)
#replacing 3+ to 3 
train['Dependents'].replace('3+', 3,inplace=True) 
test['Dependents'].replace('3+', 3,inplace=True) 
train['Loan_Status'].replace('N', 0,inplace=True) 
train['Loan_Status'].replace('Y', 1,inplace=True)

In [22]:
#Encoding categerical data in train dataset
train_onehot= pd.get_dummies(train, columns=['Gender','Married','Self_Employed','Education','Credit_History','Property_Area','Loan_Status'], prefix = ['Gender','Married','Self_Employed','Education','Credit_History','Property_Area','Loan_Status'])

train_onehot.head()


,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Self_Employed_Yes,Education_Graduate,Education_Not Graduate,Credit_History_0.0,Credit_History_1.0,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_0,Loan_Status_1
0,LP001002,0,5849,0.0,128.0,360.0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1
1,LP001003,1,4583,1508.0,128.0,360.0,0,1,0,1,...,0,1,0,0,1,1,0,0,1,0
2,LP001005,0,3000,0.0,66.0,360.0,0,1,0,1,...,1,1,0,0,1,0,0,1,0,1
3,LP001006,0,2583,2358.0,120.0,360.0,0,1,0,1,...,0,0,1,0,1,0,0,1,0,1
4,LP001008,0,6000,0.0,141.0,360.0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1


In [23]:
#Encoding categerical data in test dataset
test_onehot= pd.get_dummies(test, columns=['Gender','Married','Self_Employed','Credit_History','Education','Property_Area'], prefix = ['Gender','Married','Education','Self_Employed','Credit_History','Property_Area'])

train_onehot.head()

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Self_Employed_Yes,Education_Graduate,Education_Not Graduate,Credit_History_0.0,Credit_History_1.0,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_0,Loan_Status_1
0,LP001002,0,5849,0.0,128.0,360.0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1
1,LP001003,1,4583,1508.0,128.0,360.0,0,1,0,1,...,0,1,0,0,1,1,0,0,1,0
2,LP001005,0,3000,0.0,66.0,360.0,0,1,0,1,...,1,1,0,0,1,0,0,1,0,1
3,LP001006,0,2583,2358.0,120.0,360.0,0,1,0,1,...,0,0,1,0,1,0,0,1,0,1
4,LP001008,0,6000,0.0,141.0,360.0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1


In [28]:
X = train_onehot.drop(['Loan_Status_0','Loan_Status_1','Loan_ID'],1) 



y = train_onehot[['Loan_Status_0','Loan_Status_1']]


X.columns

Index(['Dependents', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Gender_Female', 'Gender_Male', 'Married_No',
       'Married_Yes', 'Self_Employed_No', 'Self_Employed_Yes',
       'Education_Graduate', 'Education_Not Graduate', 'Credit_History_0.0',
       'Credit_History_1.0', 'Property_Area_Rural', 'Property_Area_Semiurban',
       'Property_Area_Urban'],
      dtype='object')

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X,y, test_size =0.3)

In [30]:
model = Sequential()
model.add(Dense(64, input_dim=18, activation= "relu"))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                1216      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 130       
Total params: 9,666
Trainable params: 9,666
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss="mean_squared_error" , optimizer=SGD(lr=0.1), metrics=['accuracy'])

In [37]:
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_valid, y_valid))

Train on 429 samples, validate on 185 samples
Epoch 1/20
429/429 [==============================] - 0s 271us/step - loss: 0.3112 - acc: 0.6876 - val_loss: 0.3189 - val_acc: 0.6811
Epoch 2/20
429/429 [==============================] - 0s 29us/step - loss: 0.3124 - acc: 0.6876 - val_loss: 0.3189 - val_acc: 0.6811
Epoch 3/20
429/429 [==============================] - 0s 28us/step - loss: 0.3147 - acc: 0.6853 - val_loss: 0.3189 - val_acc: 0.6811
Epoch 4/20
429/429 [==============================] - 0s 28us/step - loss: 0.3147 - acc: 0.6853 - val_loss: 0.3189 - val_acc: 0.6811
Epoch 5/20
429/429 [==============================] - 0s 28us/step - loss: 0.3100 - acc: 0.6900 - val_loss: 0.3189 - val_acc: 0.6811
Epoch 6/20
429/429 [==============================] - 0s 27us/step - loss: 0.3077 - acc: 0.6923 - val_loss: 0.3189 - val_acc: 0.6811
Epoch 7/20
429/429 [==============================] - 0s 28us/step - loss: 0.3054 - acc: 0.6946 - val_loss: 0.3189 - val_acc: 0.6811
Epoch 8/20
429/429 [==